In [11]:
%load_ext autoreload
%autoreload 2
import os
import glob
import random
#from iucrime.records import Record

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
path = '../../data/text'
files = glob.glob(os.path.join(path, '*'))

Let's just pick one data file and take a look

In [30]:
# fname = random.choice(files)
# fname
fname = '../../data/text/1-1-14.txt'

In [31]:
print(open(fname).read())


Indiana University, Bloomington
Police Department
Student Right To Know CAD Daily Log

From Jan 01, 2014 to Jan 01, 2014.
Event #: 14-01-01-000002
Date Reported: 01/01/14 - WED at 00:31
Location : 10TH STREET/WEST OF JORDAN
Date and Time Occurred From - Occurred To
01/01/14 - WED at 00:30 - 01/01/14 - WED at 00:32
Incident : LIQUOR LAWS - ILLEGAL POSSESSION/CONSUMPTION
Report #: 140002
Disposition: CLOSED BY ARREST
Event #: 14-01-01-000003
Date Reported: 01/01/14 - WED at 00:36
Location : DELTA UPSILON
Date and Time Occurred From - Occurred To
01/01/14 - WED at 00:36 - 01/01/14 - WED at 01:13
Incident : LIQUOR LAWS - ILLEGAL POSSESSION/CONSUMPTION
Report #: 140001
Disposition: CLOSED NO ARREST
Event #: 14-01-01-000023
Date Reported: 01/01/14 - WED at 10:31
Location : ALL OTHER NON-UNIVERSITY
Date and Time Occurred From - Occurred To
01/01/14 - WED at 10:31 - 01/01/14 - WED at 10:31
Incident : NARCOTIC/DRUG LAWS - POSSESSION - MARIJUANA
Report #: 140003
Disposition: CLOSED NO ARREST
3 I

In [32]:
import re
from pprint import pprint
import datetime

class Record():
    """ class for keeping the daily IU police log """

    INFO_PATTERN = re.compile('^(location|incident|disposition|event #|report #)\s*:\s*(.+)', re.IGNORECASE)
    TIME_PATTERN = re.compile('^date and time occurred.*from.*to', re.IGNORECASE)
    
    def __init__(self, content):
        self.info = {} 
        
        self.parse(content)

    def parse(self, content):
        line_itr = iter(content.splitlines())
        for line in line_itr:
            m = re.search(Record.INFO_PATTERN, line)
            if m:
                key = m.group(1).lower().replace('#', 'number')
                val = m.group(2).lower()
                self.info[key] = val
                continue
                
            if re.match(Record.TIME_PATTERN, line):
                line = next(line_itr)
                t_from, t_to = self.parse_from_to_time(line)
                self.info['from datetime'] = t_from
                self.info['to datetime'] = t_to
                
    def parse_from_to_time(self, timestr):
        fromdate, fromtime, todate, totime = timestr.split('-')
        fromtime = fromtime.split('at')[1].strip()
        totime = totime.split('at')[1].strip()
        t_from = datetime.datetime.strptime(' '.join([fromdate.strip(), fromtime]), '%m/%d/%y %H:%M')
        t_to = datetime.datetime.strptime(' '.join([todate.strip(), totime]), '%m/%d/%y %H:%M')
        return (t_from, t_to)
                
r = Record(open(fname).read())

# assert(r.info['location'] == 'indiana memorial union')
# assert(r.info['incident'] == 'all other offenses - trespass & malicious trespass')

pprint(r.info)

{'disposition': 'closed no arrest',
 'event number': '14-01-01-000023',
 'from datetime': datetime.datetime(2014, 1, 1, 10, 31),
 'incident': 'narcotic/drug laws - possession - marijuana',
 'location': 'all other non-university',
 'report number': '140003',
 'to datetime': datetime.datetime(2014, 1, 1, 10, 31)}
